# Modelos basados en árboles


In [1]:
##--variables globales
INPUT_PATH = "/content/drive/MyDrive/data_sets/parquet/sf-airbnb-clean"

In [2]:
!pip install pyspark

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

In [4]:
#--librerias
from pyspark.ml.feature import (StringIndexer,
                                VectorAssembler)

from pyspark.ml.regression import (DecisionTreeRegressor,
                                   RandomForestRegressor)
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import (ParamGridBuilder,
                               CrossValidator)

In [6]:
airbnbDF = spark.read.parquet(INPUT_PATH)
trainDF, testDF = airbnbDF.randomSplit([.8, .2], seed=42)

### Transformación de los datos

Cuando utilizamos árboles de decisión no debemos utilizar one-hot enconding para las variables categóricas. Sin embargo, si es necesario codificarlas como variables categóricas numéricas.

In [7]:
#--Si la columna es de tipo texto, la designamos como categórica
categoricalCols = [field for (field, dataType) in trainDF.dtypes if dataType == "string"]

#--creamos los nombres de salida
indexOutputCols = [x + "Index" for x in categoricalCols]

#--definimos nuestro transformador
stringIndexer = StringIndexer(inputCols=categoricalCols,
                              outputCols=indexOutputCols,
                              handleInvalid="skip")

Combinamos las columnas categóricas codificadas y las columnas numéricas usando __VectorAssembler__

In [8]:
#--selecionamos las columnas numéricas excepto la variable respuesta
numericCols = [field for (field, dataType) in trainDF.dtypes if ((dataType == "double") & (field != "price"))]

#--combinamos los nombres de variables categóricas (salida stringIndexer) y las numéricas
assemblerInputs = indexOutputCols + numericCols

#--se combinan todos los atributos en una sola columna
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

## Árbol de decisión

El primer algoritmo que vamos a entrenar, es un árbol de desición

In [9]:
#--definimos nuestro estimador
dt_model = DecisionTreeRegressor(labelCol="price")

Combinamos todo el proceso en una Pipeline

In [11]:
#--definimos el estimador pipeline
pipeline = Pipeline(stages=[stringIndexer,
                            vecAssembler,
                            dt_model])

#--lo entrenamos
pipelineModel = pipeline.fit(trainDF)

IllegalArgumentException: requirement failed: DecisionTree requires maxBins (= 32) to be at least as large as the number of values in each categorical feature, but categorical feature 3 has 36 values. Consider removing this and other categorical features with a large number of values, or add more training examples.

### Entrenamiento

__maxBins__ determina el número de particiones que se realizan sobre las variables continuas. Este número es importante al momento de calcular las estadísticas de los atributos de forma distribuida.

Dado que tenemos variables categoricas con 36 valores, vamos a poner __maxBins__ igual a 40 y re-entrenamos el algoritmo.

In [12]:
dt_model.setMaxBins(40)

DecisionTreeRegressor_c0d0f47472c2

In [13]:
pipelineModel = pipeline.fit(trainDF)

Para visualizar el árbol decisión entrenado, lo podemos hacer de la siguiente forma:

In [14]:
last_element = pipelineModel.stages[-1]
print(last_element.toDebugString)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_c0d0f47472c2, depth=5, numNodes=47, numFeatures=33
  If (feature 12 <= 2.5)
   If (feature 12 <= 1.5)
    If (feature 5 in {1.0,2.0})
     If (feature 4 in {0.0,1.0,3.0,5.0,9.0,10.0,11.0,13.0,14.0,16.0,18.0,24.0})
      If (feature 3 in {0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0})
       Predict: 104.23992784125075
      Else (feature 3 not in {0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0})
       Predict: 250.7111111111111
     Else (feature 4 not in {0.0,1.0,3.0,5.0,9.0,10.0,11.0,13.0,14.0,16.0,18.0,24.0})
      If (feature 3 in {0.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,27.0,33.0,35.0})
       Predict: 151.94179894179894
      Else (feat

Si deseamos ver la importancia de cada atributo lo podemos ver de la siguiente forma:

In [15]:
last_element.featureImportances

SparseVector(33, {1: 0.1679, 2: 0.1401, 3: 0.0562, 4: 0.1282, 5: 0.0109, 9: 0.0388, 10: 0.0036, 12: 0.2834, 13: 0.0152, 14: 0.0295, 15: 0.1262})

Podemos utilizar el siguiente código para dar formato a la información anterior y hacer más fácil de comprender

In [16]:
import pandas as pd

#--la organizamos en un data frame
featureImp = pd.DataFrame(
    #--información
    list(zip(pipelineModel.stages[1].getInputCols(),
             last_element.featureImportances)),
    #--nombre de columnas
    columns=["feature", "importance"])

featureImp.sort_values(by="importance", ascending=False).head(10)

,feature,importance
12,bedrooms,0.283406
1,cancellation_policyIndex,0.167893
2,instant_bookableIndex,0.140081
4,property_typeIndex,0.128179
15,number_of_reviews,0.126233
3,neighbourhood_cleansedIndex,0.056200
9,longitude,0.038810
14,minimum_nights,0.029473
13,beds,0.015218
5,room_typeIndex,0.010905


### Prediccion en el train set

In [17]:
#--hacemos la prediccion del set de entrenamiento
predDF_train = (pipelineModel
          .transform(trainDF))

#--definimos un objeto para realizar la evaluación
regressionEvaluator = RegressionEvaluator(predictionCol="prediction",
                                          labelCol="price",
                                          metricName="rmse")

#--medimos el desempeño
rmse = regressionEvaluator.evaluate(predDF_train)
print(f"RMSE en set de entrenamiento: {rmse:.2f}")

RMSE en set de entrenamiento: 219.22


### Predicción test set

In [18]:
#--hacemos la prediccion del set de prueba
predDF = (pipelineModel
          .transform(testDF))

#--medimos el desempeño
rmse = regressionEvaluator.evaluate(predDF)
print(f"RMSE en set de prueba: {rmse:.2f}")

RMSE en set de prueba: 385.87


## Random Forest

In [ ]:
#--definimos nuestro estimador
rf_model = RandomForestRegressor(labelCol="price",
                                 maxBins=40,
                                 seed=42)

#--definimos todos el proceso en una pipeline, ahora usando random forest
pipe_model = Pipeline(stages = [stringIndexer,
                              vecAssembler,
                              rf_model])

### Hyperparameters: grid search

Basicamente existen dos hyperparameters que son los mas importantes en random forest:
* La profundidad de los árboles
* y el mínimo número de muestras para que una rama se divida.

Sin embargo las implementaciones de algoritmos de ML en spark son más basicas debido a la complejidad que representa realizar el algoritmo en forma distribuida.

In [ ]:
paramGrid = (ParamGridBuilder()
            .addGrid(rf_model.maxDepth, [2, 4, 6])
            .addGrid(rf_model.numTrees, [40, 60])
            .build())


### Cross validation

Para realizar la validación cruzada, necesitamos tres ingredientes:
* el algoritmo de ML: en nuestro caso la pipeline,
* un evaluador: para saber como comparar los modelos, y
* el algoritmo de validación cruzada: el que ádministrara el entrenamiento y validación de todos los modelos.

In [ ]:
#--definimos cual será la métrica para comparar cada modelo
evaluator = RegressionEvaluator(labelCol="price",
                                predictionCol="prediction",
                                metricName="rmse")

cv = CrossValidator(estimator=pipe_model, #--nuestro algoritmo de ML
                    evaluator=evaluator, #--cómo lo evaluaremos
                    estimatorParamMaps=paramGrid, #--qué parámetros probar
                    numFolds=3, #--número de particiones en nuestro training set
                    seed=42)

Lanzamos el entrenamiento:

In [ ]:
%%time
cvModel = cv.fit(trainDF)

¿Cuántos modelos acabamos de entrenar?

En grandes datos puede ser muy costoso hacer un "fine tunning" del modelo usando "cross validación" por lo que existe otra funcion que solo ajusta una vez cada combinación de parámetros:

__TrainValidationSplit__



Si queremos visualizar los resultados:

In [ ]:
list(zip(cvModel.getEstimatorParamMaps(), cvModel.avgMetrics))

## Optimizaciones

Spark entrena los modelos de la validación cruzada de forma secuencial aunque son técnicamente independientes unos de otros. Para entrenar varios modelos de forma paralela podemos utilizar el parámetro __paralellism__.

Este parámetro debe utilizarse con precaución porque si sobrepasamos los recursos del cluster, el entrenamiento resultara más lento.

Generalmente, este parámetro no deberá ser mayor a 10.

In [ ]:
%%time
cvModel = cv.setParallelism(4).fit(trainDF)

En los modelos que hemos entrenado pasamos una __pipeline__ al objeto __CrossValidator__ por tanto en cada iteración de la validación cruzada se calcula el mismo preprocesamiento de los datos (stringIndexer).

Para evitar lo anterior podemos hacer lo inverso, es decir, dar como entrada el __CrossValidator__ a una __pipeline__ y con ello preprocesar los datos una sola vez.

In [ ]:
#--definimos el objeto de cross_validation
cv = CrossValidator(estimator=rf_model,
                    evaluator=evaluator,
                    estimatorParamMaps=paramGrid,
                    numFolds=3,
                    parallelism=4,
                    seed=42)

#--nuestra pipeline, tendra el preprocesamiento de datos y al final el objeto cv
pipeline = Pipeline(stages=[stringIndexer, vecAssembler, cv])

In [ ]:
%%time
pipelineModel = pipeline.fit(trainDF)

### Predicción test set

In [ ]:
#--realizamos la predicción en el testset
#-automaticamente nuestro pipelineModel utilizara el mejor modelo
predDF = pipelineModel.transform(testDF)

#--preparamos un objeto para medir la precisión del modelo
regressionEvaluator = RegressionEvaluator(predictionCol="prediction",
                                          labelCol="price",
                                          metricName="rmse")

rmse = regressionEvaluator.evaluate(predDF)
print(f"RMSE en el test set: {rmse:.2f}")

In [ ]:
spark.stop()